In [1]:
from llama_index.core import (
PropertyGraphIndex,
SimpleDirectoryReader,
StorageContext,
Settings,
)
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore


In [2]:
llm = LlamaCPP(
model_path='models/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
temperature=0.1,
max_new_tokens=256,
context_window=4096,
model_kwargs={"n_gpu_layers": 1},
verbose=False
)
embed_model = HuggingFaceEmbedding()
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [3]:
url = "neo4j://localhost:7687"
username = ""
password = ""
database = ""
graph_store = Neo4jPropertyGraphStore(
username=username,
password=password,
url=url,
database=database,
)

In [ ]:
import nest_asyncio
import asyncio

# Apply nest_asyncio to allow nested async loops
nest_asyncio.apply()

# Your async function to create the graph index
async def create_graph_index():
    gstorage_context = StorageContext.from_defaults(graph_store=graph_store)

    documents = SimpleDirectoryReader("pdf").load_data()

    kg_index = await PropertyGraphIndex.from_documents(
        documents,
        storage_context=gstorage_context,
        max_triplets_per_chunk=10,
        include_embeddings=True,
        property_graph_store=graph_store,
    )

    return kg_index

# Handle event loop properly
loop = asyncio.get_event_loop()
kg_index = loop.run_until_complete(create_graph_index())


In [ ]:
#read the graph 
#kg_index = PropertyGraphIndex.from_existing(property_graph_store=graph_store)
